# DATASCI W261: Machine Learning at Scale 
### Name: Annie Lee

This notebook provides a poor man Hadoop through command-line and python. Please insert the python code by yourself.

# Map

In [23]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
#Please insert your code
  # Read each line in file.
  for line in myfile.readlines():
    # Convert all letters to uppercase to match "findword".
    line = line.upper()
    # Split line into words using WORD_RE regex.
    wordlist = WORD_RE.findall(line)
    # Find count of occurances of "findword".
    wordfreq = wordlist.count(findword)
    # Increment total file count.
    count += wordfreq
print count

Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

# Reduce

In [24]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for line in sys.stdin:
#Please insert your code
  # Increment overall sum by intermediate count in each line.
  sum += int(line)
print sum

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

# Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Writing pGrepCount.sh


# Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [26]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [59] [COPYRIGHT] in the file [License.txt]
